# Checking Cuda Environment

In [12]:
import torch

# Confirm that the GPU is detected

assert torch.cuda.is_available()

# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
print(f'Cuda version: {torch.version.cuda}')
device = torch.device("cuda")

Found device: GeForce GTX 1080 Ti, n_gpu: 1
Cuda version: 11.1


# Loading Pre-trained GhostNet

In [32]:
import torch
from ghostnet import ghostnet

model = ghostnet(num_classes=1000, width=1.0, dropout=0.2)
input = torch.randn(32,3,224,224)
y = model(input)
print(y)

tensor([[-0.1283, -0.2121,  0.0665,  ..., -0.2008, -0.0559,  0.1061],
        [-0.0945, -0.0554,  0.0491,  ..., -0.1715,  0.0244, -0.0206],
        [-0.1587, -0.0859,  0.0015,  ..., -0.0473, -0.0159,  0.1638],
        ...,
        [-0.0573, -0.0758,  0.1350,  ..., -0.1764,  0.0733,  0.1018],
        [-0.0007, -0.0791,  0.0384,  ..., -0.1199,  0.0648,  0.1240],
        [-0.1285, -0.0651,  0.0847,  ..., -0.1785,  0.1044,  0.0894]],
       grad_fn=<AddmmBackward>)


In [42]:
model.load_state_dict(torch.load('state_dict_73.98.pth'))
model = torch.nn.DataParallel(model, device_ids = list(range(n_gpu))).cuda()
model.eval()

DataParallel(
  (module): GhostNet(
    (conv_stem): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): GhostBottleneck(
          (ghost1): GhostModule(
            (primary_conv): Sequential(
              (0): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
            (cheap_operation): Sequential(
              (0): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
              (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): ReLU(inplace=True)
            )
          )
          (ghost2): GhostModule(
            (primary_co

In [46]:
from PIL import Image
from torchvision import transforms
import torch

imsize = 224
loader = transforms.Compose([transforms.Scale(imsize), transforms.ToTensor()])


def load_image(img_name):
    image = Image.open(img_name)
    image = loader(image).float()
    image = image.clone().detach().requires_grad_(True)
    image = image.unsqueeze(0)
    image = image.cuda()
    return image

# Tested on images from https://github.com/EliSchwartz/imagenet-sample-images
images = [load_image('ImageNetData/{0}.JPEG'.format(i)) for i in ['black_widow', 'magpie_0', 'magpie_1']]
image = images[0]
pred = model(image)
_, indices = torch.topk(pred, k=15)
print(indices)

tensor([[ 77,  73,  75,  70,  74,  72, 310, 315, 319,  79,  78, 312, 303,  76,
         120]], device='cuda:0')


# Changing Last Layer

In [ ]:
import torch.nn as nn

# Assuming you have an existing model named 'model' and want to change its last layer
# and assuming you want to change it to a new output dimension of 'num_classes'

# Get the number of input features of the last layer
num_features = model.fc.in_features

# Define the new last layer
new_last_layer = nn.Linear(num_features, num_classes)

# Assign the new last layer to the model
model.fc = new_last_layer

# Fine Tuning Model

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

# PyTorch TensorBoard support
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime


transform = transforms.Compose(
    [transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

# Create datasets for training & validation, download if necessary
training_set = torchvision.datasets.FashionMNIST('./data', train=True, transform=transform, download=True)
validation_set = torchvision.datasets.FashionMNIST('./data', train=False, transform=transform, download=True)

# Create data loaders for our datasets; shuffle for training, not for validation
training_loader = torch.utils.data.DataLoader(training_set, batch_size=4, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=4, shuffle=False)

# Class labels
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
        'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

# Report split sizes
print('Training set has {} instances'.format(len(training_set)))
print('Validation set has {} instances'.format(len(validation_set)))

loss_fn = torch.nn.CrossEntropyLoss()

# NB: Loss functions expect data in batches, so we're creating batches of 4
# Represents the model's confidence in each of the 10 classes for a given input
dummy_outputs = torch.rand(4, 10)
# Represents the correct class among the 10 being tested
dummy_labels = torch.tensor([1, 5, 3, 7])

print(dummy_outputs)
print(dummy_labels)

loss = loss_fn(dummy_outputs, dummy_labels)
print('Total loss for this batch: {}'.format(loss.item()))

optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, data in enumerate(training_loader):
        # Every data instance is an input + label pair
        inputs, labels = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Make predictions for this batch
        outputs = model(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss